# Fase 1 – Comprensión del Negocio


## 🏦 Contexto del Proyecto

Una entidad bancaria, proveniente de la India, busca ***mejorar la toma de decisiones estratégicas*** mediante el uso de sus datos históricos de clientes y transacciones.

Actualmente, el banco cuenta con registros de **clientes, transacciones y perfiles de valor**, pero carece de un sistema predictivo robusto para:

- Detectar **transacciones fraudulentas**.  
- Identificar **clientes de alto valor** para retención.  
- Segmentar clientes según su **recencia, frecuencia y valor monetario (RFM)**.  
- Estimar el **valor futuro de las transacciones** para planificación financiera.  

Esto genera **riesgos financieros** y pérdida de **oportunidades de negocio**.


## 🎯 Objetivos del Proyecto

- **General**: aprovechar datos históricos para construir un sistema de análisis predictivo y descriptivo con técnicas de Machine Learning.  

- **Específicos**:  
  - Diseñar un modelo de **clasificación** que anticipe si una transacción será fraudulenta.  
  - Construir un modelo de **regresión** para predecir el monto promedio futuro de transacciones por cliente.  
  - Aplicar segmentación basada en **RFM (Recency, Frequency, Monetary)** para agrupar clientes en perfiles de valor.  


## 📂 Datasets Seleccionados
> Ubicación: `data/01_raw/` (configurados en `conf/base/catalog.yml`)

Se seleccionaron **3 datasets principales** para el desarrollo del proyecto:

| Dataset           | Nivel | Uso en ML | Justificación |
|-------------------|-------|-----------|---------------|
| **cleaned_dataset** | Transacción | Clasificación (fraude), análisis temporal | Datos ya depurados; combina información de cliente + transacciones |
| **customer_agg**   | Cliente | Regresión (gasto futuro), features agregadas | Resume comportamiento histórico de cada cliente (totales, promedios, recencia, frecuencia) |
| **RFM**            | Cliente | Segmentación (clustering/perfiles) | Incluye puntajes y segmentos predefinidos (Champions, At Risk, etc.) |

> Nota: el dataset **bank_customer_transactions** fue descartado al estar en formato crudo y ser redundante frente a `cleaned_dataset`.


### 1) **cleaned_dataset** (nivel: **transacción**)
- **Qué contiene:** `TransactionID`, `CustomerID`, `CustomerDOB`, `CustGender`, `CustLocation`, `CustAccountBalance`, `TransactionDate`, `TransactionTime`, `TransactionAmount (INR)`, `Age`.
- **Por qué se usa:** porque contiene datos **depurados** y listos para trabajar a nivel de transacción; base ideal para **clasificación de fraude** y creación de features temporales (recencia/estacionalidad).

### 2) **customer_agg** (nivel: **cliente**)
- **Qué contiene (agregados):** `txn_count`, `total_spent`, `avg_spent`, `max_spent`, `avg_balance`, `first_txn_date`, `last_txn_date`, `location`, `gender`, `recency_days`, `tenure_days`, `txn_per_day`, `avg_spent_pct_balance`.
- **Por qué se usa:** resume el **comportamiento histórico** por cliente; útil para **regresión** (gasto futuro) y para features de modelos supervisados.

### 3) **RFM** (nivel: **cliente**)
- Este dataset contiene información relevante porque el banco busca segmentar clientes para diseñar estrategias de retención y marketing.
- **Qué contiene:** `Recency`, `Frequency`, `Monetary`, (`R`, `F` y `M`) indica sus puntajes individuales, `RFM Score` la puntuación combinada para cada cliente, (`Segment`, `Segment_Final`) corresponden a la categoría de cliente, usada en estrategias del banco.
- **Por qué se usa:** Entonces se usará porque permite **segmentar** rápidamente la base y validar perfiles (**Champions**: alta frecuencia, gasto alto, compras recientes. **At Risk**: solían comprar pero llevan mucho tiempo inactivos. **Needs Attention**: nivel intermedio, podrían ser recuperados. **Critical**: clientes con baja frecuencia, poco gasto y alta recencia).


### 4) **Significado de las variables**

| Dataset         | Variable                  | Definición                                                                 |
|-----------------|---------------------------|-----------------------------------------------------------------------------|
| cleaned_dataset | TransactionID             | Identificador único de la transacción.                                      |
| cleaned_dataset | CustomerID                | Identificador único del cliente.                                            |
| cleaned_dataset | CustomerDOB               | Fecha de nacimiento del cliente.                                            |
| cleaned_dataset | CustGender                | Género del cliente.                                                         |
| cleaned_dataset | CustLocation              | Ubicación del cliente.                                                      |
| cleaned_dataset | CustAccountBalance        | Saldo disponible al momento de la transacción.                              |
| cleaned_dataset | TransactionDate           | Fecha de la transacción.                                                    |
| cleaned_dataset | TransactionTime           | Hora de la transacción.                                                     |
| cleaned_dataset | TransactionAmount (INR)   | Monto de la transacción en rupias indias.                                   |
| cleaned_dataset | Age                       | Edad del cliente.                                                           |
| customer_agg    | txn_count                 | Número de transacciones realizadas.                                         |
| customer_agg    | total_spent               | Gasto total histórico.                                                      |
| customer_agg    | avg_spent                 | Gasto promedio por transacción.                                             |
| customer_agg    | max_spent                 | Monto máximo gastado en una transacción.                                    |
| customer_agg    | avg_balance               | Saldo promedio mantenido por el cliente.                                    |
| customer_agg    | first_txn_date            | Fecha de la primera transacción.                                            |
| customer_agg    | last_txn_date             | Fecha de la última transacción.                                             |
| customer_agg    | location                  | Ubicación del cliente.                                                      |
| customer_agg    | gender                    | Género del cliente.                                                         |
| customer_agg    | recency_days              | Días desde la última transacción.                                           |
| customer_agg    | tenure_days               | Tiempo de relación con el banco en días.                                    |
| customer_agg    | txn_per_day               | Frecuencia de transacciones por día.                                        |
| customer_agg    | avg_spent_pct_balance     | Proporción promedio del gasto respecto al saldo.                            |
| RFM             | Recency                   | Días desde la última transacción.                                           |
| RFM             | Frequency                 | Número de transacciones en un período.                                      |
| RFM             | Monetary                  | Monto total gastado en el período.                                          |
| RFM             | R                         | Puntuación de recency.                                                      |
| RFM             | F                         | Puntuación de frequency.                                                    |
| RFM             | M                         | Puntuación de monetary.                                                     |
| RFM             | RFM Score                 | Combinación de R, F y M en una sola métrica.                                |
| RFM             | Segment                   | Segmento preliminar del cliente.                                            |
| RFM             | Segment_Final             | Segmento final ajustado (*Champions, At Risk, Needs Attention, Critical*).  |


## 📌 Situación Actual

- El banco posee **grandes volúmenes de datos**, pero no los utiliza en un proceso automatizado.  
- El análisis actual es **manual y reactivo**, dificultando la detección temprana de fraude.  
- No existen modelos predictivos que permitan **anticipar riesgos** ni proyectar el **valor futuro de clientes**.  


## 🧠 Objetivos de Machine Learning

- **Clasificación** → Determinar si una transacción es **fraudulenta o legítima** (`cleaned_dataset`).  
- **Regresión** → Estimar el **monto promedio futuro** de transacciones por cliente (`customer_agg`).  
- **Segmentación (RFM)** → Generar perfiles de clientes (`RFM`) para estrategias de retención y marketing.  


## 🗺️ Plan del Proyecto (CRISP-DM)

| Semana | Actividad Principal | Entregable |
|--------|---------------------|------------|
| **1**  | Comprensión del negocio | Notebook `01_business_understanding.ipynb` |
| **2**  | Análisis exploratorio de datos (EDA) | Notebook `02_data_understanding.ipynb` |
| **3**  | Preparación y Limpieza de los datos | Notebook `03_data_preparation.ipynb` |
| **4**  | Modelado y entrega final | Repositorio GitHub con README y pipelines Kedro |
| **5**  | Documentación y entrega final | Repositorio GitHub con README y pipelines Kedro |

## 👥 Equipo de Trabajo

- 🧑‍💻 **Sebastián Carrera**  
- 🧑‍💻 **Kevin Vivanco**